In [12]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import calendar
import matplotlib.pyplot as plt

## Задание 1
Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

#### Проведём предварительное исследование данных

In [20]:
registrations = pd.read_csv('problem1-reg_data (1).csv', sep = ';')
registrations.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [21]:
def retention(df):
    
    data = df[['reg_ts','uid','auth_ts']]
    
    # Найдем минимальную и максимальную дату, с котороми может работать наша программа
    min_date = data['auth_ts'].min()
    max_date = data['auth_ts'].max()

    # Выведем на экран информацию с диапазоном дат, с которыми мы можем рабоать, для ввода даты которая нас интересует 
    period_retention = input(f'Введите дату за который вы хотите построить retention в формате год - месяц.\nВаша дата должна быть не меньше {datetime.fromtimestamp(min_date).strftime('%Y-%m')}, не больше { datetime.fromtimestamp(max_date).strftime('%Y-%m')} \n')
    
    # Сортируемый наш DataFrame по выбранному периоду 
    first_date_period = pd.to_datetime(f'{period_retention}-1')
    days_in_month = calendar.monthrange(first_date_period.year, first_date_period.month)[1]
    last_date_period = timedelta(days=days_in_month) + first_date_period

    data = data[(data['reg_ts'] >= int(first_date_period.timestamp())) & (data['auth_ts'] < int(last_date_period.timestamp()))]
    
    # Преобразуем строки с Unix-временем в читаемую дату
    data['auth_ts'] = pd.to_datetime(data['auth_ts'],unit='s').dt.normalize()
    data['reg_ts'] = pd.to_datetime(data['reg_ts'],unit='s').dt.normalize()
    
    # Рассчитаем количество дней между авторизацией и регистрацией 
    data['days'] = (data['auth_ts'] - data['reg_ts']).dt.days
    
    # Рассчитаем количество игроков на начало периода и на конец периода
    data['uid_start'] = data.groupby(['reg_ts'])['days'].transform(lambda x: (x == 0).sum())
    data['uid_end'] = data.groupby(['reg_ts', 'auth_ts'])['uid'].transform('nunique')

    # Рассчитаем retention
    data['retention'] = (data.uid_end/data.uid_start).round(2)
    
    # Убираем 1-ый день, т.к. он некорректен для расчёта
    data = data[data['days'] > 0]
    
     # Создаём сводную таблицу
    table = data.pivot_table(index = 'reg_ts', columns = 'days', values = 'retention')

    # Строим визуализацию с тепловой картой
    plt.figure(figsize=(16, 10))
    sns.heatmap(table, annot=True, cmap="YlGn", linewidth=.5)
    plt.title("Retention, %")
    plt.ylabel("Дата регистрации")
    plt.xlabel("Дней с регистрации")
    plt.tight_layout()
    plt.show()

    return table

In [22]:
retention('problem1-auth_data (1).csv', 'problem1-reg_data (1).csv')

TypeError: retention() takes 1 positional argument but 2 were given

## Если программа выдает ошибку, значит такого временного отрезка нет в документе
